In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from datetime import datetime
from datetime import timedelta
import missingno as msno

In [ ]:
# 데이터셋 로드
df =  pd.read_csv('/content/drive/MyDrive/df2.csv')

In [ ]:
df

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,division1,division2
0,2019-10-01 00:00:01+00:00,view,1307067,2053013558920217191,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook
1,2019-10-01 00:00:04+00:00,view,1004237,2053013555631882655,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone
2,2019-10-01 00:00:13+00:00,view,3900746,2053013552326770905,haier,102.38,555444559,98b88fa0-d8fa-4b9d-8a71-3dd403afab85,appliances,environment
3,2019-10-01 00:00:19+00:00,view,1306631,2053013558920217191,hp,580.89,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,computers,notebook
4,2019-10-01 00:00:19+00:00,view,1005135,2053013555631882655,apple,1747.79,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,electronics,smartphone
...,...,...,...,...,...,...,...,...,...,...
15528267,2019-10-31 23:59:58+00:00,view,2702331,2053013563911439225,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,appliances,kitchen
15528268,2019-10-31 23:59:58+00:00,view,2300275,2053013560530830019,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,electronics,camera
15528269,2019-10-31 23:59:58+00:00,view,10800172,2053013554994348409,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,unknown,unknown
15528270,2019-10-31 23:59:59+00:00,view,21407424,2053013561579406073,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,electronics,clocks


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15528272 entries, 0 to 15528271
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   event_time    object 
 1   event_type    object 
 2   product_id    int64  
 3   category_id   int64  
 4   brand         object 
 5   price         float64
 6   user_id       int64  
 7   user_session  object 
 8   division1     object 
 9   division2     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 1.2+ GB


In [ ]:
# 대분류 count 수 (abc 순)
df['division1'].value_counts().sort_index()

accessories      107890
apparel          682222
appliances      2114877
auto             437005
computers       1132918
construction     340342
country_yard       6803
electronics     6044634
furniture        569956
kids             215690
medicine           5005
sport             80031
stationery         2470
unknown         3788429
Name: division1, dtype: int64

In [ ]:
# 대분류 count 수 (내림차 순)
df['division1'].value_counts()

electronics     6044634
unknown         3788429
appliances      2114877
computers       1132918
apparel          682222
furniture        569956
auto             437005
construction     340342
kids             215690
accessories      107890
sport             80031
country_yard       6803
medicine           5005
stationery         2470
Name: division1, dtype: int64

In [ ]:
# 대분류 안에 중분류 count 수 표시
df[['division1','division2']].groupby('division1')['division2'].value_counts().to_frame()

division2
division1   division2           
accessories bag            86332
            wallet         19822
            umbrella        1736
apparel     shoes         553359
            costume        49406
...                          ...
sport       snowboard       1731
            tennis          1541
            ski             1330
stationery  cartrige        2470
unknown     unknown      3788429

[61 rows x 1 columns]

# **컨텐츠 베이스 추천시스템 제작!**

In [ ]:
# 데이터 'apparel'만 가지고 놀아봅시다
a = df[df['division1']=='apparel']
a['division2'].value_counts()

shoes        553359
costume       49406
jeans         22347
underwear     19329
dress         13287
tshirt         7574
trousers       6443
shirt          5456
scarf          1087
belt           1074
sock           1032
jumper          950
skirt           407
shorts          295
jacket          176
Name: division2, dtype: int64

In [ ]:
# 시험삼아 'tshirt'를 기준으로 CB 만들어 보기
a_model = a[a['division2'] == 'tshirt']
a_model

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,division1,division2
2130,2019-10-01 02:31:46+00:00,view,43300016,2097815753423061045,unknown,41.16,519279176,6fbf96bb-8317-46b9-8c5e-43a1ca389188,apparel,tshirt
5233,2019-10-01 02:48:16+00:00,view,43300016,2097815753423061045,unknown,41.16,512414069,be834621-86c3-4801-9e0a-cc8eba4f04fc,apparel,tshirt
7163,2019-10-01 02:57:11+00:00,view,43300022,2097815753423061045,nike,45.05,542123405,d3a56882-804e-4413-98d0-1817d5e47a24,apparel,tshirt
9148,2019-10-01 03:05:45+00:00,view,43300029,2097815753423061045,unknown,29.58,516188394,06e7844e-07ab-47bc-9e64-e312bf5182fc,apparel,tshirt
9366,2019-10-01 03:06:31+00:00,view,43300029,2097815753423061045,unknown,29.58,516188394,06e7844e-07ab-47bc-9e64-e312bf5182fc,apparel,tshirt
...,...,...,...,...,...,...,...,...,...,...
15523218,2019-10-31 22:25:01+00:00,view,43300051,2097815753423061045,nike,43.73,566266900,0f509d7d-7f61-486f-ad2e-d0c042e4659e,apparel,tshirt
15523479,2019-10-31 22:28:51+00:00,view,43300051,2097815753423061045,nike,43.73,566266900,0f509d7d-7f61-486f-ad2e-d0c042e4659e,apparel,tshirt
15523518,2019-10-31 22:29:25+00:00,view,43300051,2097815753423061045,nike,43.73,566266900,0f509d7d-7f61-486f-ad2e-d0c042e4659e,apparel,tshirt
15523560,2019-10-31 22:30:04+00:00,view,43300127,2097815753423061045,nike,45.02,566266900,0f509d7d-7f61-486f-ad2e-d0c042e4659e,apparel,tshirt


In [ ]:
# 구입한 목록이 무엇이 있는지 확인
a_model[a_model['event_type'] == 'purchase']

,event_time,event_type,product_id,category_id,brand,price,user_id,user_session,division1,division2


In [ ]:
# 데이터 전처리 (가격을 소수자리 1에서 반올림)
a_model['price'] = a_model['price'].apply(lambda x : round(x, 1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 데이터 타입 변경(가격을 숫자에서 문자로), 중복값 제거(product_id), 인덱스 초기화
a_model = a_model.astype({'product_id':'str', 'price':'str'})
a_model = a_model.drop_duplicates(['product_id'])
a_model = a_model.reset_index(drop=True)

In [ ]:
# CB에 사용할 새로운 칼럼 생성
a_model['content'] = a_model['event_type'] + '.' + a_model['brand'] + '.' + a_model['price']

In [ ]:
a_model['content']

0       view.unknown.41.2
1          view.nike.45.0
2       view.unknown.29.6
3        view.adidas.48.9
4          view.nike.45.0
              ...        
350    view.goodloot.17.2
351    view.goodloot.17.2
352    view.goodloot.17.2
353     view.carters.17.6
354        view.fila.17.8
Name: content, Length: 355, dtype: object

# **콘텐츠 기반 필터링 추천시스템**

In [ ]:
# 콘텐츠 기반 필터링 추천 분석

CB_df = a_model[['event_type','product_id','brand','price','content']]

In [ ]:
# 인덱스 초기화

CB_df = CB_df.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환.
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(CB_df['content'])
print(genre_mat.shape)


(355, 121)


In [ ]:
# 피처 벡터화된 행렬 cosine_similarities()를 적용한 코드

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])


(355, 355)
[[1.         0.2        0.6        0.2        0.2        1.
  0.6        0.2        0.2        0.6        0.2        0.6
  0.2        0.2        0.6        0.16903085 0.2        0.2
  0.6        0.2        0.2        0.2        0.2        0.2
  0.2        0.2        0.2        0.2        0.2        0.6
  0.2        0.25819889 0.25819889 0.16903085 0.2        0.2
  0.2        0.2        0.2        0.2        0.2        0.2
  0.2        0.16903085 0.6        0.2        0.2        0.2
  0.2        0.16903085 0.6        0.25819889 0.2        0.16903085
  0.16903085 0.25819889 0.25819889 0.25819889 0.2        0.2
  0.2        0.2        0.2        0.16903085 0.2        0.2
  0.6        0.2        0.2        0.2        0.25819889 0.2
  0.2        0.2        0.6        0.2        0.2        0.6
  0.25819889 0.25819889 0.25819889 0.25819889 0.2        0.16903085
  0.2        0.2        0.2        0.2        0.2        0.2
  0.2        0.2        0.2        0.2        0.2        0.16

In [ ]:

genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

# 반환된 0번 레코드를 제외하면 유사도 순으로 리스트 배열이 됨

[[  0   5  11 272  50 273 128  77  18  74  29  66  14   9 141   6  44   2
  290  56  80 271  55  78  79  57 291 279  70  81 121 100 215 176 182 194
  150 140 212 214 134 113 218 219 225 127 123 119 116  51 172  31 336  32
  307 308 311 136  17 135  16  13 137  12 133 138 139  10   8  19 131 132
  143 130 129  20  21  22 126 125 124  23 122  24 120 142  48 145 159   1
  171 170 169 167 166   3 165 164 163 162 161 160 158 146 157 156 155   4
  154 153 152 151   7 149 148 147 118  25 115 117  82  34  35  36  37  38
   76  75  39  40  73  72  71  41  69  68  67  42  65  45  64  62  61  60
   59  58  46  52  30  84  26  85  47  27 112 111 110 109 108 107 106 105
  104 102 101  28  99  98  97  96  94  93  92  91  90  89  88 173  86  87
  354 174 286 288 289 292 293 294 295 296 297 298 299 300 301 302 303 304
  305 306 287 285 310 284 263 264 175 266 267 268 269 270 274 275 276 277
  278 280 281 282 283 309 312 261 333 335 337 338 339 340 341 342 343 344
  345 346 347 348 349 350 351 352 334 

In [ ]:
CB_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event_type  355 non-null    object
 1   product_id  355 non-null    object
 2   brand       355 non-null    object
 3   price       355 non-null    object
 4   content     355 non-null    object
dtypes: object(5)
memory usage: 14.0+ KB


In [ ]:
def find_sim_product(df, sorted_ind, product_number, top_n=10):
    # 인자로 입력된 df.DataFrame에서 'product_id' 칼럼이 입력된 product_id 값인 DataFrame 추출
    product_id = df[df['product_id'] == product_number]
    
    # product_number을 가진 DataFrame의 index 객체를 ndarry로 반환하고
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = product_id.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index 출력. top_n index는 2차원 데이터임.
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [ ]:
similar_products = find_sim_product(CB_df, genre_sim_sorted_ind, '43300022', 10)
similar_products = similar_products.reset_index(drop=True)
similar_products[['product_id', 'brand']]

# 코사인 유사도에 따른 10개의 품목이 추천이됨. / content 칼럼 안에 : event, brand, price 순으로 기록됨

[[132   1 154 130   4 147 133  48 251 306]]


,product_id,brand
0,43300130,nike
1,43300022,nike
2,43300128,nike
3,43300126,nike
4,43300023,nike
5,43300127,nike
6,43300129,nike
7,43300021,nike
8,43300103,nike
9,43300101,nike


# **위의 과정을 압축한 함수화 진행**

In [ ]:

# 데이터와 해당 품목에 대한 전처리를 시작합니다.
def CB_Making(df, division1_value, division2_value):
        b = df
        b = df[df['division1']== division1_value]
        b_model = b[b['division2'] == division2_value]
        b_model['price'] = b_model['price'].apply(lambda x : round(x, 1))
        b_model = b_model.astype({'product_id':'str', 'price':'str'})
        b_model = b_model.drop_duplicates(['product_id'])
        b_model = b_model.reset_index(drop=True)
        b_model['content'] = b_model['event_type'] + '.' + b_model['brand'] + '.' + b_model['price']
        CB2_df = b_model[['event_type','product_id','brand','price','content']]
        CB2_df = CB2_df.reset_index(drop=True)
        return CB2_df

# CB 추출
def CB_Show(CB2_df, product_number):
        count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
        genre_mat = count_vect.fit_transform(CB2_df['content'])
        genre_sim = cosine_similarity(genre_mat, genre_mat)
        genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
        similar_products = find_sim_product(CB2_df, genre_sim_sorted_ind, product_number, 10)
        similar_products = similar_products.reset_index(drop=True)
        return similar_products[['product_id', 'brand']]


In [ ]:
c = CB_Making(df, 'accessories', 'bag') # accessories와 bag의 추천시스템

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
CB_Show(c,'32900146') #product_id 32900146의 추천시스템

[[1459 1477 1516 1492 1491 1478 1460 1523 1461 1500]]


,product_id,brand
0,32900146,salmo
1,32900147,salmo
2,32900158,salmo
3,32900159,salmo
4,32900161,salmo
5,32900148,salmo
6,32900149,salmo
7,32900150,salmo
8,32900151,salmo
9,32900153,unknown
